# Modules, fichiers, expressions régulières (correction)

Correction.

In [4]:
%matplotlib inline

### Exercice 1 : Excel $\rightarrow$ Python $\rightarrow$ Excel
    
On récupère le fichier [seance4_excel.csv](https://github.com/sdpython/teachpyx/raw/main/_data/seance4_excel.csv) :

- le lire sous python
- créer une matrice carrée 3x3 où chaque valeur est dans sa case (X,Y),
- enregistrer le résultat sous format texte,
- le récupérer sous Excel.    

On utilise le module `urllib` pour télécharger automatiquement le fichier :

In [6]:
from urllib.request import urlopen

url = "https://github.com/sdpython/teachpyx/raw/main/_data/seance4_excel.csv"
filename = "seance4_excel.csv"
with urlopen(url) as u:
    with open(filename, "wb") as f:
        f.write(u.read())

In [8]:
with open("seance4_excel.csv", "r") as f:
    mat = [row.strip(" \n").split(",") for row in f.readlines()]

mat = mat[1:]
res = [[None] * 3 for i in range(5)]
for i, j, v in mat:
    res[int(j) - 1][int(i) - 1] = float(v)

with open("seance4_excel_mat.csv", "w") as f:
    f.write("\n".join([",".join([str(x) for x in row]) for row in res]))

Il est très rare d'écrire ce genre de code. En règle générale, on se sert de modules déjà existant comme [pandas](http://pandas.pydata.org/). Cela évite la conversion au format texte :

In [11]:
import pandas

df = pandas.read_csv("seance4_excel.csv")
mat = df.pivot(index="X", columns="Y", values="value")
mat.to_csv("seance4_excel_mat.csv")
mat

Y,1,2,3,4,5
X,,,,,
1,0.0,-1.0,-2.0,-3.0,-4.0
2,1.0,0.0,-1.0,-2.0,-5.0
3,2.0,1.0,0.0,-1.0,NaN


C'est un peu plus rapide.

<h3 id="exo2">Exercice 2 : trouver un module (1)</h3>

Le module [random](https://docs.python.org/3.4/library/random.html) est celui qu'on cherche.

In [12]:
import random

alea = [random.random() for i in range(10)]
print(alea)
random.shuffle(alea)
print(alea)

[0.6414749515069115, 0.4096411389655439, 0.7991926327688407, 0.9918119068265183, 0.07379598399157106, 0.4290244325276996, 0.9337274118746318, 0.17483162769873706, 0.6159652170653446, 0.4835251137894401]
[0.4835251137894401, 0.4096411389655439, 0.6414749515069115, 0.9918119068265183, 0.7991926327688407, 0.07379598399157106, 0.17483162769873706, 0.9337274118746318, 0.6159652170653446, 0.4290244325276996]


### Exercice 3 : trouver un module (2)

Le module [datetime](https://docs.python.org/3.4/library/datetime.html) permet de faire des opérations sur les dates.

In [13]:
from datetime import datetime

date1 = datetime(2013, 9, 9)
date0 = datetime(2013, 8, 1)
print(date1 - date0)
birth = datetime(1975, 8, 11)
print(birth.weekday())  # lundi

39 days, 0:00:00
0


### Exercice 4 : son propre module

On effectue le remplacement ``if __name__ == "__main__":``  par ``if True :`` :

In [14]:
# fichier monmodule2.py
import math


def fonction_cos_sequence(seq):
    return [math.cos(x) for x in seq]


if __name__ == "__main__":
    # et une petite astuce quand on travaille sous notebook
    code = """
        # coding: utf-8
        import math
        def fonction_cos_sequence(seq) :
            return [ math.cos(x) for x in seq ]        
        if True :
            print ("Ce message n'apparaît que si ce programme est le point d'entrée.")
        """.replace(
        "        ", ""
    )
    with open("monmodule3.py", "w", encoding="utf8") as f:
        f.write(code)

In [15]:
import monmodule3

print(monmodule3.fonction_cos_sequence([1, 2, 3]))

Ce message n'apparaît que si ce programme est le point d'entrée.
[0.5403023058681398, -0.4161468365471424, -0.9899924966004454]


Le message ``ce message n'apparaît que ce programme est le point d'entrée`` apparaît maintenant alors qu'il n'apparaissait pas avec la version de l'énoncé. Comme il apparaît après ``*``, cela montre que cette ligne est exécutée si le module est importé.

In [16]:
import monmodule3

Si on importe le module une seconde fois, le message n'apparaît plus : le langage Python a détecté que le module avait déjà été importé. Il ne le fait pas une seconde fois.

### Exercice 5 : chercher un motif dans un texte

L'expression régulière est ``je .{1,60}``. Le symbol ``.`` signifie n'importe quel caractère. Suivi de ``{1,60}`` veut dire n'importe quel caractère répété entre 1 et 60 fois.

In [18]:
from teachpyx.tools.data_helper import download_and_unzip

url = "https://github.com/sdpython/teachpyx/raw/main/_data/voeux.zip"
discours = download_and_unzip(url)
discours

['VOEUX01.txt',
 'VOEUX05.txt',
 'VOEUX06.txt',
 'VOEUX07.txt',
 'VOEUX08.txt',
 'VOEUX09.txt',
 'VOEUX74.txt',
 'VOEUX75.txt',
 'VOEUX79.txt',
 'VOEUX83.txt',
 'VOEUX87.txt',
 'VOEUX89.txt',
 'VOEUX90.txt',
 'VOEUX94.txt']

On affiche le contenu du premier fichier. L'encoding définit la façon dont son représentés les caractères accentuées. L'usage veut que ce soit `utf-8` mais cette convention est récente et les anciens fichiers ne la suivent pas toujours. Pour deviner la valeur de ce paramètre, on peut utiliser le module [chardet](https://github.com/chardet/chardet) :

In [28]:
import chardet

with open("VOEUX01.txt", "rb") as f:
    content = f.read()
print(chardet.detect(content))

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [25]:
with open("VOEUX01.txt", "r", encoding="ISO-8859-1") as f:
    print(f.read()[:1000])

 Mes chers compatriotes,

Les moments où un peuple se rassemble, respire au même rythme, bat d'un même coeur, sont toujours des moments privilégiés. Les fêtes de Noël et du Nouvel An participent de ces rendez-vous du pays avec lui-même. Pour beaucoup d'entre vous, c'est un temps de joie, de retrouvailles, de chaleur familiale et amicale. Pour certains, parce qu'ils sont seuls, malades ou dans la peine, c'est une période difficile. Et j'ai pour eux ce soir une pensée particulière. Et je pense aussi à nos compatriotes de Toulouse, mais également de la Somme et de l'est de la France, pour qui le souvenir de 2001 restera d'abord celui des épreuves. A chacune et à chacun d'eux, je veux dire, en ce 31 décembre, la solidarité et l'amitié de la nation. Et à vous tous, mes chers compatriotes de métropole, d'Outre-mer, de l'étranger, j'adresse mes voeux très chaleureux de bonne et heureuse année.

*

Avec l'arrivée de l'euro, nous allons vivre dans quelques heures un moment historique : 300 mill

In [29]:
import re

exp = re.compile("je .{1,60}", re.IGNORECASE)
for fichier in discours:
    print("----", fichier)
    try:
        with open(fichier, "r") as f:
            text = f.read()
    except:
        with open(fichier, "r", encoding="ISO-8859-1") as f:
            text = f.read()
    je = exp.findall(text)
    for t in je:
        print(t)

---- VOEUX01.txt
je pense aussi à nos compatriotes de Toulouse, mais également d
je veux dire, en ce 31 décembre, la solidarité et l'amitié de l
je sais que vous voulez faire vivre les valeurs qui sont celles
je souhaite une bonne et heureuse année.
---- VOEUX05.txt
je pense d'abord à toutes celles et à tous ceux qui connaissent
Je pense aussi à nos soldats qui sont engagés sur tous les cont
je veux dire à la famille de Bernard PLANCHE, que les pouvoirs 
je prendrai rapidement des initiatives pour relancer la constru
---- VOEUX06.txt
je pense d'abord à toutes celles et à tous ceux qui sont victim
Je veux aussi saluer avec respect et reconnaissance nos soldats
je me bats pour qu'elle prenne toute sa place dans ce nouveau m
Je me bats pour que chaque Français, pour que chaque jeune en p
je voudrais que les choses avancent plus vite, et qu'elles avan
je m'y engagerai pleinement. Je voudrais vous dire ce soir quel
Je sais les souffrances et les difficultés auxquelles certains 
Je connais v

### Exercice 6 : chercher un autre motif dans un texte

Pour les mots _securite_ ou _insecurite_, on construit l'expression :

In [30]:
exp = re.compile("(.{1,15}(in)?sécurité.{1,50})", re.IGNORECASE)
for fichier in discours:
    print("----", fichier)
    try:
        with open(fichier, "r") as f:
            text = f.read()
    except:
        with open(fichier, "r", encoding="latin-1") as f:
            text = f.read()
    je = exp.findall(text)
    for t in je:
        print(t)

---- VOEUX01.txt
("enacée quand l'insécurité progresse, quand l'exercice des libertés, la qual", 'in')
---- VOEUX05.txt
---- VOEUX06.txt
("traites, notre sécurité sociale se réforment et c'est la garantie de leur", '')
("rvice, pour la sécurité, pour l'emploi, pour le pouvoir d'achat et que ce", '')
(" l'énergie, la sécurité, l'immigration. N'oublions jamais que l'Europe, c", '')
---- VOEUX07.txt
---- VOEUX08.txt
('vie pour notre sécurité et pour la paix. Je veux penser à leurs familles ', '')
---- VOEUX09.txt
('garantir notre sécurité.', '')
---- VOEUX74.txt
---- VOEUX75.txt
---- VOEUX79.txt
---- VOEUX83.txt
("tres : plus de sécurité, des banlieues rénovées et plus d'enfants dans no", '')
---- VOEUX87.txt
---- VOEUX89.txt
(' de paix et de sécurité.', '')
---- VOEUX90.txt
(' du Conseil de sécurité, organe suprême des Nations unies. A ce titre, no', '')
(' du Conseil de sécurité, y compris le recours éventuel à la force. Voilà ', '')
(" d'Israël à la sécurité, ni le droit des Palestinie

### Exercice 7 : recherche les urls dans une page wikipédia

On pourra prendre comme exemple la page du programme [Python](https://fr.wikipedia.org/wiki/Python_(langage)). La première partie consiste à récupérer le contenu d'une page HTML.

In [31]:
from urllib.request import urlopen

url = "https://fr.wikipedia.org/wiki/Python_(langage)"
with urlopen(url) as u:
    content = u.read()
content[:300]

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-fe'

Les données récupérées sont au format binaire d'où le préfixe ``b''``. Pour éviter de télécharger les données à chaque fois, on sauve le contenu sur disque pour le récupérer la prochaine fois.

In [32]:
with open("page.html", "wb") as f:
    f.write(content)

Et on le recharge.

In [34]:
with open("page.html", "rb") as f:
    page = f.read()
page[:300]

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-fe'

Les données sont sous forme d'octets, il faut d'abord les convertir sous forme de caractères. il y a plus de caractères que d'octets disponibles (256), c'est cela qu'il faut une sorte de code pour passer de l'un à l'autre : dans le cas d'internet, le plus utilisé est l'[encoding utf-8](https://fr.wikipedia.org/wiki/UTF-8).

In [35]:
page_str = page.decode("utf-8")
page_str[:300]

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-fe'

On recherche maintenant les urls commençant par http...

In [36]:
import re

reg = re.compile('href=\\"(http.*?)\\"')
urls = reg.findall(page_str)
urls[:10]

['https://fr.wikipedia.org/wiki/Python_(langage)',
 'https://creativecommons.org/licenses/by-sa/4.0/deed.fr',
 'https://af.wikipedia.org/wiki/Python_(programmeertaal)',
 'https://als.wikipedia.org/wiki/Python_(Programmiersprache)',
 'https://an.wikipedia.org/wiki/Python',
 'https://ar.wikipedia.org/wiki/%D8%A8%D8%A7%D9%8A%D8%AB%D9%88%D9%86_(%D9%84%D8%BA%D8%A9_%D8%A8%D8%B1%D9%85%D8%AC%D8%A9)',
 'https://as.wikipedia.org/wiki/%E0%A6%AA%E0%A6%BE%E0%A6%87%E0%A6%A5%E0%A6%A8',
 'https://ast.wikipedia.org/wiki/Python',
 'https://az.wikipedia.org/wiki/Python_(proqramla%C5%9Fd%C4%B1rma_dili)',
 'https://azb.wikipedia.org/wiki/%D9%BE%D8%A7%DB%8C%D8%AA%D9%88%D9%86']

### Exercice 8 : construire un texte à motif

A l'inverse des expressions régulières, des modules comme [Mako](http://www.makotemplates.org/) ou [Jinja2](http://jinja.pocoo.org/docs/dev/) permettent de construire simplement des documents qui suivent des règles. Ces outils sont très utilisés pour la construction de page web. On appelle cela faire du [templating](https://wiki.python.org/moin/Templating). Créer une page web qui affiche à l'aide d'un des modules la liste des dimanches de cette année.

In [37]:
patron = """
<ul>{% for i, url in enumerate(urls) %}
  <li><a href="{{ url }}">url {{ i }}</a></li>{% endfor %}
</ul>
"""

In [38]:
from jinja2 import Template

tpl = Template(patron)

In [39]:
print(tpl.render(urls=urls[:10], enumerate=enumerate))


<ul>
  <li><a href="https://fr.wikipedia.org/wiki/Python_(langage)">url 0</a></li>
  <li><a href="https://creativecommons.org/licenses/by-sa/4.0/deed.fr">url 1</a></li>
  <li><a href="https://af.wikipedia.org/wiki/Python_(programmeertaal)">url 2</a></li>
  <li><a href="https://als.wikipedia.org/wiki/Python_(Programmiersprache)">url 3</a></li>
  <li><a href="https://an.wikipedia.org/wiki/Python">url 4</a></li>
  <li><a href="https://ar.wikipedia.org/wiki/%D8%A8%D8%A7%D9%8A%D8%AB%D9%88%D9%86_(%D9%84%D8%BA%D8%A9_%D8%A8%D8%B1%D9%85%D8%AC%D8%A9)">url 5</a></li>
  <li><a href="https://as.wikipedia.org/wiki/%E0%A6%AA%E0%A6%BE%E0%A6%87%E0%A6%A5%E0%A6%A8">url 6</a></li>
  <li><a href="https://ast.wikipedia.org/wiki/Python">url 7</a></li>
  <li><a href="https://az.wikipedia.org/wiki/Python_(proqramla%C5%9Fd%C4%B1rma_dili)">url 8</a></li>
  <li><a href="https://azb.wikipedia.org/wiki/%D9%BE%D8%A7%DB%8C%D8%AA%D9%88%D9%86">url 9</a></li>
</ul>
